# **CrowdAI submission**

In [12]:
#Imports
import numpy as np
import pandas as pd
import random
import math
import plots
from matplotlib import pyplot as plt
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import SVDpp
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import SlopeOne
from surprise import accuracy
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from sklearn.model_selection import KFold as skFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from Vince_helpers import *

my_seed = 200
random.seed(my_seed)
np.random.seed(my_seed)

In [14]:
#Load data train file
train = pd.read_csv('data_train.csv')
train = to_surprise(train)

In [15]:
#Load train set into Surprise
reader = Reader(rating_scale=(1, 5))
train_surp = Dataset.load_from_df(train, reader)
train_surp = train_surp.build_full_trainset()

Fit each model with preselected hyperparameters

In [18]:
mean = global_mean(train)
users = user_mean(train)
movies = movie_mean(train)

In [20]:
algo_baseline = BaselineOnly().fit(train_surp)
algo_SVDb = SVD().fit(train_surp)
algo_SVD = SVD().fit(train_surp)
#algo_SVDpp = SVDpp().fit(train_surp)
algo_slope_one = SlopeOne().fit(train_surp)
#algo_knn_user = KNNBaseline().fit(train_surp)
algo_knn_movie = KNNBaseline().fit(train_surp)

Estimating biases using als...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


Load user/movie pairs to predict

In [21]:
test_original = pd.read_csv('examples_sample_submission.csv')

In [22]:
#Move testset to surprise testset format
test = test_original.copy()
test = to_surprise(test)
test = Dataset.load_from_df(test, reader)
test = test.build_full_trainset()
test = test.build_testset()

In [25]:
#Predict ratings with every model
predictions_baseline = algo_baseline.test(test)
predictions_SVDb = algo_SVDb.test(test)
predictions_SVD = algo_SVD.test(test)
#predictions_SVDpp = algo_SVDpp.test(test)
predictions_slope_one = algo_slope_one.test(test)
#predictions_knn_user = algo_knn_user.test(test)
#predictions_knn_movie = algo_knn_movie.test(test)

CPU times: user 7.56 s, sys: 307 ms, total: 7.87 s
Wall time: 7.88 s
CPU times: user 11 s, sys: 326 ms, total: 11.3 s
Wall time: 11.4 s
CPU times: user 10.6 s, sys: 262 ms, total: 10.8 s
Wall time: 10.8 s
CPU times: user 2min 40s, sys: 848 ms, total: 2min 40s
Wall time: 2min 41s


In [26]:
#Extract estimated ratings
uids = [pred.uid for pred in predictions_baseline]
mids = [pred.iid for pred in predictions_baseline]
ruis = [pred.r_ui for pred in predictions_baseline]
est_baseline = [pred.est for pred in predictions_baseline]
est_SVDb = [pred.est for pred in predictions_SVDb]
est_SVD = [pred.est for pred in predictions_SVD]
#est_SVDpp = [pred.est for pred in predictions_SVDpp]
est_slope_one = [pred.est for pred in predictions_slope_one]
#est_knn_user = [pred.est for pred in predictions_knn_user]
#est_knn_movie = [pred.est for pred in predictions_knn_movie]
est_global = [mean for i in range(len(ruis))]
est_user_mean = [predict_user(u, users, mean) for u in uids]
est_movie_mean = [predict_movie(m, movies, mean) for m in mids]

In [49]:
#Build Dataframe containing ratings predictions
est_baseline = np.array(est_baseline)
est_global = np.array(est_global)
est_user_mean = np.array(est_user_mean)
est_movie_mean = np.array(est_movie_mean)
#est_knn_movie = np.array(est_knn_movie)
#est_knn_user = np.array(est_knn_user)
est_slope_one = np.array(est_slope_one)
est_SVDb = np.array(est_SVDb)
est_SVD = np.array(est_SVD)
#est_SVDpp = np.array(est_SVDpp)

'''
X = np.column_stack((est_global, est_user_mean, est_movie_mean, est_baseline, 
                     est_knn_movie, est_knn_user, est_slope_one,
                     est_SVDb, est_SVD, est_SVDpp))
'''

X = np.column_stack((est_global, est_user_mean, est_movie_mean, est_baseline, 
                     est_slope_one, est_SVDb, est_SVD))

In [ ]:
#Linear combination of predictions
weights = np.array([0.1428]*7)
#weights = np.array([0.1]*10)
preds = X.dot(weights)
#Clip interval to 1-5 and round predictions to nearest integer
preds = np.clip(preds, 1, 5)
preds = np.around(preds)

In [51]:
#Recover proper ids format
ids = np.array(['r'+str(u)+'_c'+str(m) for (u,m) in zip(uids, mids)])

Create submission file

In [52]:
sub = pd.DataFrame({'Id':ids, 'Prediction':preds})

In [55]:
sub.to_csv('subVince', index=False)